In [14]:
import numpy as np

# Option parameters
S0 = 100  # Initial stock price
K = 100  # Strike price
T = 1.0  # Time to maturity in years
r = 0.05  # Risk-free interest rate
q = 0.02  # Dividend yield
sigma = 0.2  # Volatility
num_simulations = 1000000  # Number of simulations

# Seed for reproducibility
np.random.seed(42)

# Simulate end-of-period stock prices
Z = np.random.standard_normal(num_simulations)  # Standard normal random variables
ST = S0 * np.exp(
    ((r - q) - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z
)  # Stock price at maturity

# Calculate the payoff at maturity
c_payoff = np.maximum(ST - K, 0)
p_payoff = np.maximum(K - ST, 0)

# Discount the payoff back to present value
call_option_price = np.exp(-r * T) * np.mean(c_payoff)
put_option_price = np.exp(-r * T) * np.mean(p_payoff)

call_option_price, put_option_price

(9.211944296853586, 6.34585373876795)

In [15]:
import numpy as np

def price_american_option(S0, K, T, r, sigma, q, num_simulations, num_steps, option_type='put'):
    # Time parameters
    dt = T / num_steps
    discount_factor = np.exp(-r * dt)

    # Simulate stock price paths
    S = np.zeros((num_simulations, num_steps + 1))
    S[:, 0] = S0
    for t in range(1, num_steps + 1):
        Z = np.random.standard_normal(num_simulations)
        S[:, t] = S[:, t-1] * np.exp((r - q - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # Initialize the payoff matrix
    if option_type == 'put':
        payoff = np.maximum(K - S, 0)
    elif option_type == 'call':
        payoff = np.maximum(S - K, 0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    # Initialize the cash flow matrix
    cash_flow = payoff[:, -1]

    # Backward induction to estimate the continuation value
    for t in range(num_steps - 1, 0, -1):
        in_the_money = payoff[:, t] > 0
        # Perform regression only on in-the-money paths
        X = S[in_the_money, t]
        Y = cash_flow[in_the_money] * discount_factor
        if len(X) == 0:
            continue
        # Basis functions (polynomials)
        A = np.vstack([np.ones_like(X), X, X**2]).T
        # Least squares regression
        coeffs = np.linalg.lstsq(A, Y, rcond=None)[0]
        continuation_value = np.dot(A, coeffs)
        exercise_value = payoff[in_the_money, t]
        # Optimal stopping decision
        exercise = exercise_value > continuation_value
        cash_flow[in_the_money] = np.where(exercise, exercise_value, cash_flow[in_the_money] * discount_factor)

    # Discount the final cash flows back to present value
    option_price = np.mean(cash_flow * discount_factor)

    return option_price

# Option parameters
S0 = 100        # Initial stock price
K = 100         # Strike price
T = 1.0         # Time to maturity in years
r = 0.05        # Risk-free interest rate
sigma = 0.2     # Volatility
q = 0.02        # Continuous dividend yield
num_simulations = 100000  # Number of simulations
num_steps = 200  # Number of time steps

# Seed for reproducibility
np.random.seed(42)

american_put_price = price_american_option(S0, K, T, r, sigma, q, num_simulations, num_steps, option_type='put')
american_call_price = price_american_option(S0, K, T, r, sigma, q, num_simulations, num_steps, option_type='call')

american_call_price, american_put_price


(9.102387179561426, 6.702017934570528)

In [16]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

def price_american_option_decision_tree(S0, K, T, r, sigma, q, num_simulations, num_steps, option_type=None):
    # Time parameters
    dt = T / num_steps
    discount_factor = np.exp(-r * dt)

    # Simulate stock price paths
    S = np.zeros((num_simulations, num_steps + 1))
    S[:, 0] = S0
    for t in range(1, num_steps + 1):
        Z = np.random.standard_normal(num_simulations)
        S[:, t] = S[:, t-1] * np.exp((r - q - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # Initialize the payoff matrix
    if option_type == 'Put':
        payoff = np.maximum(K - S, 0)
    elif option_type == 'Call':
        payoff = np.maximum(S - K, 0)
    else:
        raise ValueError("option_type must be 'Call' or 'Put'")

    # Initialize the cash flow matrix
    cash_flow = payoff[:, -1]

    # Backward induction to estimate the continuation value
    for t in range(num_steps - 1, 0, -1):
        in_the_money = payoff[:, t] > 0
        # Perform regression only on in-the-money paths
        X = S[in_the_money, t].reshape(-1, 1)
        Y = cash_flow[in_the_money] * discount_factor
        if len(X) == 0:
            continue
        
        # Use DecisionTreeRegressor to estimate continuation value
        tree = DecisionTreeRegressor(max_leaf_nodes=10)
        tree.fit(X, Y)
        continuation_value = tree.predict(X)
        
        exercise_value = payoff[in_the_money, t]
        # Optimal stopping decision
        exercise = exercise_value > continuation_value
        cash_flow[in_the_money] = np.where(exercise, exercise_value, cash_flow[in_the_money] * discount_factor)

    # Discount the final cash flows back to present value
    option_price = np.mean(cash_flow * discount_factor)

    return option_price

# Option parameters
S0 = 100        # Initial stock price
K = 100         # Strike price
T = 1.0         # Time to maturity in years
r = 0.05        # Risk-free interest rate
sigma = 0.2     # Volatility
q = 0.02        # Continuous dividend yield
num_simulations = 100000  # Number of simulations
num_steps = 50  # Number of time steps

# Seed for reproducibility
np.random.seed(42)

# Price the American put option
american_put_price = price_american_option_decision_tree(S0, K, T, r, sigma, q, num_simulations, num_steps, option_type='Put')
american_call_price = price_american_option_decision_tree(S0, K, T, r, sigma, q, num_simulations, num_steps, option_type='Call')

american_call_price, american_put_price


(8.032532944701792, 6.389913396399722)